In [2]:
import os, sys, copy

import numpy as np
import pandas as pd

from fractions import Fraction
from tabulate import tabulate

from helper import *

In [3]:
# no. of decision variables (N)
NO_OF_DECISION_VARS = 2

# no. of conditions 
NO_OF_CONDITIONS = 2

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [1, 1]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, 'operation', <solution>]
# the operation can be one of the '>=', '<=', or '='
conditions = [
    [   3, 2, 5],
    [   0, 1, 2],
]

In [4]:
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"

# ========================= ALL THE VARIABLES =========================
var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("variables:", var_symbol_arr)

# ========================== BASIC VARIABLES ==========================
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_CONDITIONS):
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("basic variables:", basic_var_symbol_arr)

variables: ['P', 'X1', 'X2', 'S1', 'S2']
basic variables: ['P', 'S1', 'S2']


In [5]:
# verify the elements
OBJ_FUNC = np.array(OBJ_FUNC)
if len(OBJ_FUNC) != NO_OF_DECISION_VARS:
    raise Exception(f"the no. of coefficients in the objective function doesn't match tne number of decision variables...")

conditions = np.array(conditions)
if len(conditions) != NO_OF_CONDITIONS:
    raise Exception(f"the no. of specified conditions doesn't match 'NO_OF_CONDITIONS'...")
if len(conditions[0]) != NO_OF_DECISION_VARS + 1:
    raise Exception(f"the no. of coefficients in the conditions doesn't match tne number of decision variables...")

In [6]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# construct the initial tabulation
obj_func_row = np.concatenate(([1], -OBJ_FUNC, np.zeros([NO_OF_CONDITIONS]), [0]))
initial_tabulation = obj_func_row.reshape((1, -1))

for i in range(1, col_size):

    # condition rows
    temp = np.zeros([NO_OF_CONDITIONS])
    temp[i-1] = 1
    row = np.concatenate(([0], conditions[i-1][ :-1], temp, conditions[i-1][-1: ])).reshape((1, -1))

    initial_tabulation = np.concatenate((initial_tabulation, row), axis=0)

# convert all the elements into fractions
for i in range(initial_tabulation.shape[0]):
    initial_tabulation[i] = np.array(list(map(Fraction, initial_tabulation[i])))

visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   S1 │   S2 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │   -1 │   -1 │    0 │    0 │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │    3 │    2 │    1 │    0 │     5 │
├────┼─────┼──────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │    0 │    1 │    0 │    1 │     2 │
└────┴─────┴──────┴──────┴──────┴──────┴───────┘


In [7]:
initial_tabulation

array([[ 1., -1., -1.,  0.,  0.,  0.],
       [ 0.,  3.,  2.,  1.,  0.,  5.],
       [ 0.,  0.,  1.,  0.,  1.,  2.]])

In [8]:
integer_program(initial_tabulation, var_symbol_arr, basic_var_symbol_arr, OPT_TYPE)

AttributeError: 'numpy.float64' object has no attribute 'denominator'